In [1]:
"""! pip install numpy pandas seaborn matplotlib torch transformers sklearn captum plotly emoji transformers_interpret"""

'! pip install numpy pandas seaborn matplotlib torch transformers sklearn captum plotly emoji transformers_interpret'

In [30]:
import os, sys
import random
import argparse
from pathlib import Path
import logging
from torch import cuda
from tqdm import tqdm 
import pickle

from collections import Counter

import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# from flair.data import Sentence
# from flair.models import SequenceTagger

import spacy

# from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoModel
from transformers import AdamW, get_linear_schedule_with_warmup
# from transformers_interpret import SequenceClassificationExplainer
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification


# %% Loading custom libraries 
sys.path.append('../metrics/')
# from performance import f1_score_func, accuracy_per_class

# Loading the custom library
sys.path.append('../process/')
from load_data import FetchData, ContextualizedData
from utils import merge_and_create_dataframe, train_model, evaluate_model, clean_and_merge_data_for_tokenization, add_tokens_to_vocabulary

#from captum.attr import visualization as viz
#from captum.attr import LayerConductance, LayerIntegratedGradients

In [31]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [32]:
args = dict(ads_count=20,
 batch_size=64,
 cuda=False,
 data='alpha-dreams',
 data_dir='../data',
 delta=0.01,
 dropout=0.65,
 early_stopping=True,
 eval_per_steps=2000,
 hidden_states=512,
 load_model='epoch_38.model',
 lr=4e-05,
 max_seq_len=512,
 mode='train',
 model='bert',
 n_splits=5,
 nb_epochs=10,
 patience=3,
 preprocess_flag=False,
 save_dir='../models/merged',
 seed=1111,
 setting='high',
 split_ratio=0.25,
 version='full')

In [33]:
# %% Loading the datasets
alpha_df = pd.read_csv(os.path.join(args['data_dir'], "preprocessed_alpha.csv"), error_bad_lines=False, 
                            lineterminator='\n', usecols=['marketplace', 'title', 'vendor', 'prediction', 'ships_to', 'ships_from', 'description']).drop_duplicates()
dreams_df = pd.read_csv(os.path.join(args['data_dir'], "preprocessed_dreams.csv"), error_bad_lines=False, 
                            lineterminator='\n', usecols=['marketplace', 'title', 'vendor', 'prediction', 'ships_to', 'ships_from', 'description']).drop_duplicates()
silk_df = pd.read_csv(os.path.join(args['data_dir'], "preprocessed_silk.csv"), error_bad_lines=False, 
                            lineterminator='\n', usecols=['marketplace', 'title', 'vendor', 'prediction', 'ships_to', 'ships_from', 'description']).drop_duplicates()
agora_df = pd.read_csv(os.path.join(args['data_dir'], "preprocessed_agora.csv"), error_bad_lines=False, encoding = "ISO-8859-1")
agora_df = agora_df[['Vendor', ' Item', ' Item Description']]
agora_df.columns = ['vendor', 'title', 'description']
agora_df['vendor'] = agora_df['vendor'].apply(lambda x : str(x).lower())

data_df = {"alpha":alpha_df, "dreams":dreams_df, "silk":silk_df, "agora":agora_df}

In [34]:
dream_df = dreams_df[dreams_df['marketplace']=='dream']

In [35]:
valhalla_df = dreams_df[dreams_df['marketplace']=='valhalla']
traderoute_df = dreams_df[dreams_df['marketplace']=='traderoute']
berlusconi_df = dreams_df[dreams_df['marketplace']=='berlusconi']

In [36]:
[(train_alpha_dreams, train_dreams_silk, train_alpha_silk, train_alpha_dreams_silk, train_alpha, train_dreams, train_silk), (test_alpha_dreams, test_dreams_silk, test_alpha_silk, test_alpha_dreams_silk, test_alpha, test_dreams, test_silk)] = FetchData(data_df, args["version"], args["data"],  args["split_ratio"], args["preprocess_flag"], args["setting"], args["ads_count"],  args["seed"]).split_data()

Cleaning the data ...
Training and Test data size for Alphabay market : (75327, 7) (25109, 7)
Training and Test data size for Dreams market : (70198, 7) (23400, 7)
Training and Test data size for Silk-Road market : (89544, 7) (29848, 7)
Splitting combined data ...


In [37]:
[(train_valhalla_traderoute, train_traderoute_berlusconi, train_valhalla_berlusconi, train_traderoute_agora, train_valhalla_traderoute_berlusconi, train_valhalla, train_traderoute, train_berlusconi, train_agora), (test_valhalla_traderoute, test_traderoute_berlusconi, test_valhalla_berlusconi, test_traderoute_agora, test_valhalla_traderoute_berlusconi, test_valhalla, test_traderoute, test_berlusconi, test_agora)] = FetchData(data_df, 'full', 'traderoute-agora',  args["split_ratio"], args["preprocess_flag"], "low", args["ads_count"],  args["seed"]).split_data()

Cleaning the data ...
Training and Test data size for Valhalla market : (1631, 7) (544, 7)
Training and Test data size for Traderoute market : (14964, 7) (4989, 7)
Training and Test data size for Berlusconi market : (1077, 7) (360, 7)
Training and Test data size for Agora market : (82266, 3) (27423, 3)
Splitting combined data ...


# Evaluating trained model in zero shot setting

In [204]:
# Vectorizing class labels
# data = pd.concat([train_dreams, test_dreams, train_alpha, test_alpha, train_silk, test_silk])
data = pd.concat([train_dreams, test_dreams])
all_vendors = list(data['vendor'].unique())
vendor_to_idx_dict = {vendor_name:index for index, vendor_name in enumerate(all_vendors)}

In [222]:
a = list(vendor_to_idx_dict.keys())

In [224]:
a.sort()

In [226]:
print(a)

['0hp1', '0ldamsterdamm', '24drugstore', '24k.', '25-7', '2gd4dn', '2good4dn', '420dispensary', '420highstreet', '420medicine', '5starhaze', '911turbo', '9incecloud', 'a1crack', 'ablonelsd', 'abracadabrauk', 'achlysuk', 'adderallz', 'aeirla', 'afrojack', 'agorameds', 'airforceone', 'ajaxfc', 'akindercare', 'alancuring420_uk', 'alaurizen', 'albert-hofmann', 'albertheijn', 'alex_jones', 'alibabauk', 'aliexpressss', 'alisdrugstore', 'allgoldeverything', 'allroids', 'alphabeast', 'alphacvv', 'alwaysoverweight', 'amazon-crime', 'amazonprime', 'americangreed', 'americansteroids', 'amphetamine', 'amsterdam-finest', 'amsterdam2015', 'amsterdam2017', 'amsterdam_inc', 'amsterdamconnection', 'amsterdamfinest', 'amsterdamflex', 'amsterdaminc', 'amsterdamnl', 'amsterdamquality', 'anabolic_supplies', 'anabolicwindow', 'andychem', 'angelina', 'anonsheep', 'antarcticastore', 'antonsen', 'apbay100', 'aphelion', 'apkallu', 'apotheke', 'appleinc', 'aracay', 'artvandelay', 'ashlii', 'atacori', 'atmosphere

In [214]:
common_vendors = list(set(agora_df.vendor.unique()).intersection(set(vendor_to_idx_dict.keys())))
len(common_vendors)

113

In [215]:
common_df = agora_df.loc[agora_df['vendor'].isin(common_vendors)]
# common_df = merge_and_create_dataframe(common_df)

In [216]:
common_df.shape

(25635, 3)

# Alpha-Dreams

In [8]:
# Vectorizing class labels
# data = pd.concat([train_dreams, test_dreams, train_alpha, test_alpha, train_silk, test_silk])
data = pd.concat([train_dreams, test_dreams])
all_vendors = list(data['vendor'].unique())
vendor_to_idx_dict = {vendor_name:index for index, vendor_name in enumerate(all_vendors)}

In [36]:
# df_train = pd.concat([train_alpha, train_silk])
# df_test = pd.concat([test_alpha, test_silk])
df_train = train_dreams
df_test = test_dreams
# vendors = [vendor if vendor in vendor_to_idx_dict.keys() else 'others' for vendor in df_test['vendor']]
# df_test['vendor'] = vendors

df_train['vendor'] = df_train['vendor'].replace(vendor_to_idx_dict, regex=True)
df_test['vendor'] = df_test['vendor'].replace(vendor_to_idx_dict, regex=True)

# train_df = merge_and_create_dataframe(train_dreams)
# test_df = merge_and_create_dataframe(test_dreams)
train_df = merge_and_create_dataframe(df_train).drop_duplicates()
test_df = merge_and_create_dataframe(df_test).drop_duplicates()

# Valhalla-Berlusconi

In [38]:
data = pd.concat([train_valhalla, test_valhalla, train_berlusconi, test_berlusconi])
all_vendors = list(data['vendor'].unique())
vendor_to_idx_dict = {vendor_name:index for index, vendor_name in enumerate(all_vendors)}

In [ ]:
# df_train = pd.concat([train_dreams, train_alpha, train_silk])
# df_test = pd.concat([test_dreams, test_alpha, test_silk])

df_train = pd.concat([train_valhalla, train_berlusconi])
df_test = pd.concat([test_valhalla, test_berlusconi])

df_train['vendor'] = df_train['vendor'].replace(vendor_to_idx_dict, regex=True)
df_test['vendor'] = df_test['vendor'].replace(vendor_to_idx_dict, regex=True)

# train_df = merge_and_create_dataframe(train_dreams)
# test_df = merge_and_create_dataframe(test_dreams)
train_df = merge_and_create_dataframe(df_train).drop_duplicates()
test_df = merge_and_create_dataframe(df_test).drop_duplicates()

# Traderoute-Agora

In [10]:
data = pd.concat([train_traderoute, test_traderoute, train_agora, test_agora])
all_vendors = list(data['vendor'].unique())
vendor_to_idx_dict = {vendor_name:index for index, vendor_name in enumerate(all_vendors)}

In [ ]:
df_train = pd.concat([train_traderoute, train_agora])
df_test = pd.concat([test_traderoute, test_agora])

df_train['vendor'] = df_train['vendor'].replace(vendor_to_idx_dict, regex=True)
df_test['vendor'] = df_test['vendor'].replace(vendor_to_idx_dict, regex=True)

# train_df = merge_and_create_dataframe(train_dreams)
# test_df = merge_and_create_dataframe(test_dreams)
train_df = merge_and_create_dataframe(df_train).drop_duplicates()
test_df = merge_and_create_dataframe(df_test).drop_duplicates()

# Adapter model

In [21]:
"""from datasets import load_dataset

data_files = {"train": "train.csv", "test": "test.csv", "valid":"valid.csv"}
print("Loading the Merged dataset .... ")
dataset = load_dataset("Vageesh/merged", data_files=data_files, use_auth_token="api_azAmDxuDpyfUgNahyXDcJjAOKVeTUkHkdZ")"""

'from datasets import load_dataset\n\ndata_files = {"train": "train.csv", "test": "test.csv", "valid":"valid.csv"}\nprint("Loading the Merged dataset .... ")\ndataset = load_dataset("Vageesh/merged", data_files=data_files, use_auth_token="api_azAmDxuDpyfUgNahyXDcJjAOKVeTUkHkdZ")'

In [22]:
"""from transformers import BertTokenizer, BertConfig, BertModelWithHeads
# Initializing the model
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=len(vendor_to_idx_dict), )
model = BertModelWithHeads.from_pretrained("bert-base-uncased", config=config, )"""

'from transformers import BertTokenizer, BertConfig, BertModelWithHeads\n# Initializing the model\nconfig = BertConfig.from_pretrained("bert-base-uncased", num_labels=len(vendor_to_idx_dict), )\nmodel = BertModelWithHeads.from_pretrained("bert-base-uncased", config=config, )'

In [23]:
"""# load model
model = BertModel.from_pretrained('../models/merged/adapt-bert/checkpoint-299500/')
model.load_adapter('../models/merged/adapt-bert/checkpoint-299500/')"""

"# load model\nmodel = BertModel.from_pretrained('../models/merged/adapt-bert/checkpoint-299500/')\nmodel.load_adapter('../models/merged/adapt-bert/checkpoint-299500/')"

# Loading the Bert base model

In [40]:
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification

In [41]:
# from ipywidgets import FloatProgress

In [42]:
from transformers import BertTokenizer, BertForSequenceClassification
# Load the BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, do_lower_case=True)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                            num_labels=len(vendor_to_idx_dict),
                                            output_attentions=False,
                                            output_hidden_states=False).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [154]:
"""model = BertForSequenceClassification.from_pretrained("../models/uncased/bert/epoch_1.model", 
                                                        num_labels=len(vendor_to_idx_dict),
                                                        output_attentions=False,
                                                        output_hidden_states=False).to(device)"""

'model = BertForSequenceClassification.from_pretrained("../models/uncased/bert/epoch_1.model", \n                                                        num_labels=len(vendor_to_idx_dict),\n                                                        output_attentions=False,\n                                                        output_hidden_states=False).to(device)'

In [43]:
# load model
model.load_state_dict(torch.load("../models/merged/valhalla-berlusconi/bert/epoch_14.model"))
model.eval()
model.zero_grad()

In [96]:
"""from transformers import RobertaTokenizer, RobertaForSequenceClassification
# Load the RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                            num_labels=len(vendor_to_idx_dict),
                                            output_attentions=False,
                                            output_hidden_states=False).to(device)"""

from transformers import ElectraTokenizer, ElectraForSequenceClassification
# Load the Electra tokenizer and model
tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator', truncation=True, do_lower_case=True)
model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator',
                                            num_labels=len(vendor_to_idx_dict),
                                            output_attentions=False,
                                            output_hidden_states=False).to(device)


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [15]:
# load model
model.load_state_dict(torch.load("../models/merged/traderoute-agora/electra/epoch_40.model"))
model.eval()
model.zero_grad()

# Loading Distill Bert

In [50]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
# Load the Distill bert tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                            num_labels=len(vendor_to_idx_dict),
                                            output_attentions=False,
                                            output_hidden_states=False).to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [56]:
# load model
model.load_state_dict(torch.load("../models/merged/valhalla-berlusconi/distill/epoch_10.model"))
model.eval()
model.zero_grad()

In [45]:
len(vendor_to_idx_dict)

1422

# Evaluating trained model

In [46]:
# test_df = pd.concat([test_alpha, test_dreams, test_silk])

In [54]:
def create_data_for_evaluation(test_data):
    # vendors = [vendor if vendor in vendor_to_idx_dict.keys() else 'others' for vendor in test_data['vendor']]
    # test_data['vendor'] = vendors
    # test_data['vendor'] = test_data['vendor'].replace(vendor_to_idx_dict, regex=True)
    # test_df = merge_and_create_dataframe(test_data)

    encoded_data_test = tokenizer.batch_encode_plus(test_data.text.values, add_special_tokens=True, return_attention_mask=True, 
                                                    pad_to_max_length=True, max_length=512, return_tensors='pt')
    input_ids_test = encoded_data_test['input_ids']
    attention_masks_test = encoded_data_test['attention_mask']
    labels_test = torch.tensor(list(test_data.labels.values))

    dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)
    dataloader_test = DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=32)

    return dataloader_test

In [58]:
test_df = pd.concat([test_valhalla_berlusconi])

In [59]:
vendors = [vendor if vendor in vendor_to_idx_dict.keys() else 'others' for vendor in test_df['vendor']]

In [60]:
test_df['vendor'] = vendors
test_df['vendor'] = test_df['vendor'].replace(vendor_to_idx_dict, regex=True)
test_df = merge_and_create_dataframe(test_df)

In [61]:
# Evaluating on the Alphabay dataset
dataloader_test = create_data_for_evaluation(test_df)
_, predictions, true_vals = evaluate_model(model, dataloader_test, device)

/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Evaluating model ....


In [62]:
print(sklearn.metrics.classification_report(np.array(test_df.labels), torch.argmax(torch.tensor(predictions), dim=1).numpy(), digits=4))

              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000         9
           1     1.0000    1.0000    1.0000         3
           2     1.0000    1.0000    1.0000         6
           3     1.0000    1.0000    1.0000         4
           4     1.0000    1.0000    1.0000         6
           5     1.0000    1.0000    1.0000         2
           6     1.0000    0.8333    0.9091         6
           7     1.0000    1.0000    1.0000         1
           8     0.9405    0.9651    0.9527       344
           9     1.0000    1.0000    1.0000         9
          10     1.0000    1.0000    1.0000         4
          11     1.0000    1.0000    1.0000         3
          12     1.0000    1.0000    1.0000         4
          13     1.0000    1.0000    1.0000         2
          14     1.0000    1.0000    1.0000        14
          15     1.0000    1.0000    1.0000         1
          16     1.0000    1.0000    1.0000         5
          17     0.8182    

/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1

In [20]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

68509928

# Helper Functions

In [183]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [184]:
def predict(inputs):
    return model(inputs)[0]

In [185]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = model.bert.embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = model.bert.embeddings(ref_input_ids, token_type_ids=ref_token_type_ids, position_ids=ref_position_ids)
    
    return input_embeddings, ref_input_embeddings

In [186]:
def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0][1].unsqueeze(-1)

In [187]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [188]:
lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

In [189]:
"""def generate_visualization(df, idx, vendor_to_idx_dict):
    print("Vendor name:", list(vendor_to_idx_dict.keys())[list(vendor_to_idx_dict.values()).index(idx)])
    text = df[df['text']==idx]
    
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    
    attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=700,
                                    internal_batch_size=3,
                                    return_convergence_delta=True)
    
    attributions_sum = summarize_attributions(attributions)
    
    score_vis = viz.VisualizationDataRecord(attributions_sum,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 1)[0]),
                                        df['labels'].iloc[idx],
                                        text,
                                        attributions_sum.sum(),       
                                        all_tokens,
                                        delta)
    
    print('\033[1m', 'Visualization For Score', '\033[0m')
    viz.visualize_text([score_vis])
    return score, attributions_sum"""

'def generate_visualization(df, idx, vendor_to_idx_dict):\n    print("Vendor name:", list(vendor_to_idx_dict.keys())[list(vendor_to_idx_dict.values()).index(idx)])\n    text = df[df[\'text\']==idx]\n    \n    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)\n    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)\n    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)\n    attention_mask = construct_attention_mask(input_ids)\n\n    indices = input_ids[0].detach().tolist()\n    all_tokens = tokenizer.convert_ids_to_tokens(indices)\n    \n    attributions, delta = lig.attribute(inputs=input_ids,\n                                    baselines=ref_input_ids,\n                                    n_steps=700,\n                                    internal_batch_size=3,\n                                    return_convergence_delta=True)\n    \n    attributions_sum = summa

In [190]:
vendor = list(vendor_to_idx_dict.keys())[20]
vendor_id = vendor_to_idx_dict[vendor]

In [192]:
# One can test a couple of examples and check that the sentiment classifier is behaving
text = sample_df['text'].iloc[24]
text
#text = "The movie was one of those crappy movies you can't forget.

'Title : starEmoji  instahearts || instagram success starEmoji \n Description : (censored due to possible presence of pii)'

In [193]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [194]:
# Check predict output
predict(input_ids)

tensor([[ 6.9189, -0.6540, -0.1456,  ...,  2.3671, -1.0465, -2.8723]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [195]:
# Check output of custom_forward
custom_forward(input_ids)

tensor([7.2332e-11], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [196]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    n_steps=1000,
                                    internal_batch_size=3,
                                    return_convergence_delta=True)

In [197]:
score = predict(input_ids)

print('Advertisement: ', text)
print('Prediction: ' + str(torch.argmax(score[0]).cpu().numpy()) + \
      ', Probability positive: ' + str(torch.softmax(score, dim = 1)[0][1].cpu().detach().numpy()))



Advertisement:  Title : starEmoji  instahearts || instagram success starEmoji 
 Description : (censored due to possible presence of pii)
Prediction: 20, Probability positive: 7.233172e-11


In [217]:
torch.softmax(score, dim = 1)[0]

tensor([1.4067e-07, 7.2332e-11, 1.2026e-10,  ..., 1.4839e-09, 4.8851e-11,
        7.8690e-12], device='cuda:0', grad_fn=<SelectBackward>)

In [198]:
prediction_probs = list(score[0].cpu().detach().numpy())

In [199]:
attributions_sum = summarize_attributions(attributions)

In [200]:
attributions_sum

tensor([ 0.0000e+00, -5.3472e-02, -2.0227e-01,  1.8990e-01, -8.4392e-02,
        -2.9028e-01, -1.7298e-01, -4.0119e-02,  2.9922e-01, -3.5510e-02,
        -5.7739e-02, -1.8759e-01, -1.1684e-01, -1.7097e-04,  3.0305e-01,
        -2.9476e-01, -1.2952e-02, -4.4370e-01,  1.3903e-01,  4.2050e-02,
        -2.2160e-01, -5.5021e-02, -2.7620e-01, -1.2343e-01,  2.5250e-02,
         5.1196e-02,  4.1175e-02,  6.4433e-03, -6.5567e-02,  8.1828e-02,
         2.2770e-02,  3.1185e-01,  0.0000e+00], device='cuda:0',
       dtype=torch.float64)

In [201]:
token_list, attribution_list = ([] for i in range(2))
for idx, token in enumerate(all_tokens):
    if "#" not in token:
        token_list.append(token)
        attribution_list.append(attributions_sum[idx])
    else:
        token_list[-1] += token.replace("##","")
        attribution_list[-1] += attributions_sum[idx]

attribution_list = torch.tensor(attribution_list)

In [212]:
score_vis = viz.VisualizationDataRecord(attributions_sum,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 1)[0]),
                                        sample_df['labels'].iloc[2],
                                        text,
                                        attributions_sum.sum(),       
                                        token_list,
                                        delta)

In [213]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji instahearts || instagram success starEmoji Description : (censored due to possible presence of pii),-2.21,[CLS] title : staremoji instahearts | | instagram success staremoji description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji instahearts || instagram success starEmoji Description : (censored due to possible presence of pii),-2.21,[CLS] title : staremoji instahearts | | instagram success staremoji description : ( censored due to possible presence of pii ) [SEP]


In [93]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : kingPieceEmoji paypal / ebay transfer method kingPieceEmoji Description : (censored due to possible presence of pii),-2.59,[CLS] title : kingpieceemoji paypal / ebay transfer method kingpieceemoji description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : kingPieceEmoji paypal / ebay transfer method kingPieceEmoji Description : (censored due to possible presence of pii),-2.59,[CLS] title : kingpieceemoji paypal / ebay transfer method kingpieceemoji description : ( censored due to possible presence of pii ) [SEP]


In [105]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : kingPieceEmoji multiple ways to make money online kingPieceEmoji Description : (censored due to possible presence of pii),-0.56,[CLS] title : kingpieceemoji multiple ways to make money online kingpieceemoji description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : kingPieceEmoji multiple ways to make money online kingPieceEmoji Description : (censored due to possible presence of pii),-0.56,[CLS] title : kingpieceemoji multiple ways to make money online kingpieceemoji description : ( censored due to possible presence of pii ) [SEP]


In [122]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji porn accountstarEmoji jizzonteens – lifetime Description : (censored due to possible presence of pii),-2.07,[CLS] title : staremoji porn accountstaremoji jizzonteens – lifetime description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji porn accountstarEmoji jizzonteens – lifetime Description : (censored due to possible presence of pii),-2.07,[CLS] title : staremoji porn accountstaremoji jizzonteens – lifetime description : ( censored due to possible presence of pii ) [SEP]


In [138]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji instahearts || instagram success starEmoji Description : (censored due to possible presence of pii),-2.21,[CLS] title : staremoji instahearts | | instagram success staremoji description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji instahearts || instagram success starEmoji Description : (censored due to possible presence of pii),-2.21,[CLS] title : staremoji instahearts | | instagram success staremoji description : ( censored due to possible presence of pii ) [SEP]


In [153]:
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(model,tokenizer)

In [162]:
# One can test a couple of examples and check that the sentiment classifier is behaving
text = sample_df['text'].iloc[0]
text

'Title : kingPieceEmoji  bank and paypal cashout with video kingPieceEmoji \n Description : (censored due to possible presence of pii)'

In [163]:
word_attributions = cls_explainer(text)
print(word_attributions)
all_tokens = [token for (token, attribution) in word_attributions]
all_attributions = [attribution for (token, attribution) in word_attributions]

[('[CLS]', 0.0), ('title', 0.0021046673678678253), (':', 0.041838146675838656), ('king', 0.10049682539379798), ('##piece', 0.09746727482201298), ('##em', 0.110747435114034), ('##oj', 0.209761232713998), ('##i', 0.3170736407300666), ('bank', -0.06845975443838256), ('and', -0.09489295977044694), ('pay', 0.0022840341332830286), ('##pal', -0.0017140839983914586), ('cash', 0.1490077051857321), ('##out', 0.07869648654741095), ('with', 0.024431973592838108), ('video', 0.020016347548299014), ('king', 0.08358778948649748), ('##piece', 0.1426965623451249), ('##em', 0.1972250629327529), ('##oj', 0.12336607595397744), ('##i', 0.26030187609038463), ('description', 0.12879453383492623), (':', 0.10225753701545438), ('(', 0.3093147225077939), ('ce', 0.07580953162722846), ('##nsor', 0.4702827561144549), ('##ed', 0.2823009026196751), ('due', 0.3175758422487204), ('to', 0.14862154244461265), ('possible', 0.09143839048455309), ('presence', 0.16406632744458763), ('of', 0.09350405974083252), ('pi', 0.027080

In [165]:
token_list, attribution_list = ([] for i in range(2))
for idx, token in enumerate(all_tokens):
    if "#" not in token:
        token_list.append(token)
        attribution_list.append(all_attributions[idx])
    else:
        token_list[-1] += token.replace("##","")
        attribution_list[-1] += all_attributions[idx]

attribution_list = torch.tensor(attribution_list)

In [166]:
attribution_list

tensor([ 0.0000e+00,  2.1047e-03,  4.1838e-02,  8.3555e-01, -6.8460e-02,
        -9.4893e-02,  5.6995e-04,  2.2770e-01,  2.4432e-02,  2.0016e-02,
         8.0718e-01,  1.2879e-01,  1.0226e-01,  3.0931e-01,  8.2839e-01,
         3.1758e-01,  1.4862e-01,  9.1438e-02,  1.6407e-01,  9.3504e-02,
         1.3052e-01,  1.4766e-01,  0.0000e+00])

In [167]:
score_vis = viz.VisualizationDataRecord(attribution_list,
                                        torch.softmax(score, dim = 1)[0][1],
                                        torch.argmax(torch.softmax(score, dim = 1)[0]),
                                        sample_df['labels'].iloc[2],
                                        text,
                                        sum(attribution_list),       
                                        token_list,
                                        delta)

In [147]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji instahearts || instagram success starEmoji Description : (censored due to possible presence of pii),3.99,[CLS] title : staremoji instahearts | | instagram success staremoji description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : starEmoji instahearts || instagram success starEmoji Description : (censored due to possible presence of pii),3.99,[CLS] title : staremoji instahearts | | instagram success staremoji description : ( censored due to possible presence of pii ) [SEP]


In [157]:
cls_explainer.visualize("distilbert_negative_attr.html")


In [168]:
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : kingPieceEmoji bank and paypal cashout with video kingPieceEmoji Description : (censored due to possible presence of pii),4.26,[CLS] title : kingpieceemoji bank and paypal cashout with video kingpieceemoji description : ( censored due to possible presence of pii ) [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
20,20 (0.00),Title : kingPieceEmoji bank and paypal cashout with video kingPieceEmoji Description : (censored due to possible presence of pii),4.26,[CLS] title : kingpieceemoji bank and paypal cashout with video kingpieceemoji description : ( censored due to possible presence of pii ) [SEP]


In [164]:
cls_explainer.visualize("distilbert_negative_attr.html")
